In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import time

In [2]:
def random_k(rnd):
    return np.power(1 - rnd, -2/3)

In [3]:
num_samples = 1000
observations_per_sample = np.power(10, np.arange(1, 6+1))

In [ ]:
samples = [None] * observations_per_sample.shape[0]

for n in range(len(samples)):
    t0 = time.time()
    observations = np.round(random_k(np.random.random(size=(num_samples, observations_per_sample[n]))))
    t1 = time.time()
    print(n, t1-t0)

0 0.0008208751678466797
1 0.0051882266998291016
2 0.051284074783325195
3 0.43097591400146484
4 4.620021104812622
